In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import google_streetview.api as goos

c:\users\user\appdata\local\programs\python\python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
c:\users\user\appdata\local\programs\python\python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
c:\users\user\appdata\local\programs\python\python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
c:\users\user\appdata\local\programs\python\python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
c:\users\user\appdata\local\programs\python\python37\lib\importl

Outline: 
* data sourcing
    * set up google api to download images
    * generate random lat, long
    * test location for being in urban area covered by map
    * use street view to find nearest image
    * get image metadata inc lat lon & direction
    * see if there is sidewalk centerline w/in 10ft?, create label in dataframe
* modeling
    * create tf model to classify images, train/test/validate
* use pretrained model to do object detection of sidewalks
* cluster sidewalks by type
    

In [1]:
pwd

'C:\\Users\\User\\Documents\\GitHub\\sidewalks_deep-learning'

In [8]:
#download data source if not there
from os import path
if not path.exists("sidewalks.geojson"):
    import urllib.request
    data_source_url=r'https://gis.drcog.org/geoserver/DRCOGPUB/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=DRCOGPUB:planimetrics_2016_centerline_sidewalks&outputFormat=application%2Fjson'
    urllib.request.urlretrieve(data_source_url, 'sidewalks.geojson')

In [10]:
import geopandas as gpd

In [11]:
sidewalk_gdf = gpd.read_file('sidewalks.geojson')

In [12]:
sidewalk_gdf.head()

,id,gid,createdate,createuser,type,comments,update_sta,unique_id,geometry
0,planimetrics_2016_centerline_sidewalks.534414,534414,2016-04-08,Kucera,Sidewalk,None,N,534414,"MULTILINESTRING ((3096575.759 1779104.505, 309..."
1,planimetrics_2016_centerline_sidewalks.534415,534415,2016-04-08,Kucera,Sidewalk,None,N,534415,"MULTILINESTRING ((3095429.059 1779035.067, 309..."
2,planimetrics_2016_centerline_sidewalks.534416,534416,2016-04-08,Kucera,Other crossing,None,N,534416,"MULTILINESTRING ((3096759.611 1779106.863, 309..."
3,planimetrics_2016_centerline_sidewalks.534417,534417,2016-04-08,Kucera,Sidewalk,None,N,534417,"MULTILINESTRING ((3096821.734 1779106.976, 309..."
4,planimetrics_2016_centerline_sidewalks.534418,534418,2016-04-08,Kucera,Sidewalk,None,N,534418,"MULTILINESTRING ((3096759.611 1779106.863, 309..."


In [16]:
sidewalk_gdf.dtypes

id              object
gid              int64
createdate      object
createuser      object
type            object
comments        object
update_sta      object
unique_id        int64
geometry      geometry
dtype: object

In [18]:
import google_streetview.api as gsv
import os

In [26]:
#Dev key is unique to each user. To run this notebook, create your own dev key and store in directory in a text file of this name.
!google_streetview -s key="google_dev_key.key"


 Set "key" default to "google_dev_key.key"


In [31]:
# Import google_streetview for the api module
import google_streetview as gsv
# Import google_streetview for the api and helper module

# Create a dictionary with multiple parameters separated by ;
apiargs = {
  'location': '46.414382,10.013988;40.720032,-73.988354',
  'size': '640x300;640x640',
  'heading': '0;90;180;270',
  'fov': '0;90;120',
  'pitch': '-90;0;90'
}

# Get a list of all possible queries from multiple parameters
api_list = gsv.helpers.api_list(apiargs)

# Create a results object for all possible queries
results = gsv.api.results(api_list)

# Preview results
results.preview()

# Download images to directory 'downloads'
results.download_links('downloads')

# Save metadata
results.save_metadata('metadata.json')

KeyError: 'pano_id'